In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts
import hvplot.pandas
import panel as pn
from bokeh.io import output_notebook

output_notebook()
hv.extension('bokeh')
pn.extension('bokeh')


Loading BokehJS ...

/tmp/ipykernel_14310/806704305.py:14: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('bokeh')


In [2]:
monkey = 'fiona' # 'yasmin'  or 'fiona' 
base_path = Path.cwd().parent / 'data' / f'{monkey}_sst'
filepath = base_path.parent / 'csst_trials_pkls' / f'{monkey}_csst_trials_df.pkl'
df = pd.read_pickle(filepath)

In [3]:
print(df.info())
# df.iloc[:2]
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110358 entries, 0 to 110357
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   blinks                  21339 non-null   object 
 1   dir                     110358 non-null  int64  
 2   direction               110358 non-null  object 
 3   filename                110358 non-null  object 
 4   first_relevant_saccade  103093 non-null  object 
 5   go_cue                  110358 non-null  int64  
 6   hPos                    110358 non-null  object 
 7   hVel                    110358 non-null  object 
 8   neural_data             102343 non-null  object 
 9   saccades                110206 non-null  object 
 10  segs_durations          110358 non-null  object 
 11  segs_times              110358 non-null  object 
 12  set                     110358 non-null  object 
 13  speed                   110358 non-null  object 
 14  ssd_len             

,blinks,dir,direction,filename,first_relevant_saccade,go_cue,hPos,hVel,neural_data,saccades,...,ssd_number,stop_cue,trail_number,trail_session,trial_failed,trial_length,trial_name,type,vPos,vVel
0,None,180,L,fi210824a.1272,"[1340, 1413]",1074,"[-11.0, -11.0, -11.0, -11.0, -11.0, -10.975, -...","[-1.6540165034091119, -1.6540165034091119, -3....","{1: [1254.93], 5: [31.33, 813.0, 827.33, 914.9...","[[136, 217], [702, 749], [1340, 1413], [1383, ...",...,NaN,NaN,1272,fi210824a,False,2225,GO_L,GO,"[-1.475, -1.475, -1.475, -1.475, -1.475, -1.47...","[-2.389134949368717, -2.389134949368717, -2.66..."
1,None,0,R,fi210824a.0935,"[1248, 1321]",1054,"[11.275, 11.275, 11.25, 11.275, 11.275, 11.275...","[2.7566941723485194, 2.7566941723485194, 3.216...","{1: [415.6], 2: [892.9, 1326.28, 1338.55, 1832...","[[155, 237], [1248, 1321], [1296, 1334]]",...,3.0,1234.0,0935,fi210824a,False,2205,CONT_R_SSD3,CONT,"[-0.55, -0.55, -0.55, -0.55, -0.55, -0.55, -0....","[-0.5513388344697039, -0.5513388344697039, -1...."
2,None,0,R,fi210824a.0478,"[1367, 1424]",970,"[2.975, 2.975, 2.975, 2.975, 2.975, 2.9, 2.9, ...","[-15.069928142171907, -15.069928142171907, -13...","{1: [1093.62, 1552.1699999999998], 2: [164.57,...","[[0, 96], [70, 141], [402, 460], [1367, 1424]]",...,1.0,1054.0,0478,fi210824a,False,1754,STOP_R_SSD1,STOP,"[-18.225, -18.225, -17.2, -16.65, -16.65, -15....","[188.098432359914, 188.098432359914, 188.09843..."
3,"[179, 291]",180,L,fi210824a.1313,"[1152, 1226]",1016,"[11.575, 11.575, 11.55, 11.55, 11.55, 11.575, ...","[-1.286457280429309, -1.286457280429309, -2.29...","{0: [537.97, 1364.45], 3: [895.95, 1309.28], 4...","[[132, 208], [271, 413], [388, 456], [648, 711...",...,2.0,1148.0,1313,fi210824a,True,1849,STOP_L_SSD2,STOP,"[-0.775, -0.775, -0.775, -0.775, -0.775, -0.75...","[-0.8270082517045559, -0.8270082517045559, -2...."
4,None,180,L,fi210824a.1067,"[1417, 1496]",1046,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.025, -0.025,...","[1.6540165034091119, 1.6540165034091119, 1.654...","{1: [493.8], 4: [924.67], 5: [358.08, 976.1800...","[[619, 659], [1417, 1496]]",...,2.0,1178.0,1067,fi210824a,False,2197,CONT_L_SSD2,CONT,"[0.9, 0.9, 0.9, 0.875, 0.875, 0.85, 0.85, 0.85...","[-0.09188980574495066, -0.09188980574495066, -..."


In [4]:
# Neural Population Analysis - Behavioral Exploratory Data Analysis

## Cell 1: Data Loading and Basic Info
print(f"Data loaded for {monkey}")
print(f"Total trials: {len(df):,}")
print(f"Date range: {df['trail_session'].str[:8].min()} to {df['trail_session'].str[:8].max()}")


Data loaded for fiona
Total trials: 110,358
Date range: fi210628 to fi211125


In [5]:
## Cell 2: Basic Data Overview

# Basic overview of behavioral data
print("=== BEHAVIORAL DATA OVERVIEW ===")
print(f"\nTrial Types:")
print(df['type'].value_counts().sort_index())

print(f"\nDirections:")
print(df['direction'].value_counts())

print(f"\nTrial Outcomes:")
success_rate = (1 - df['trial_failed'].mean()) * 100
print(f"Overall Success Rate: {success_rate:.1f}%")
print(df['trial_failed'].value_counts())

print(f"\nExperimental Set:")
print(df['set'].value_counts())




=== BEHAVIORAL DATA OVERVIEW ===

Trial Types:
type
CONT    25294
GO      61460
STOP    23604
Name: count, dtype: int64

Directions:
direction
R    55453
L    54905
Name: count, dtype: int64

Trial Outcomes:
Overall Success Rate: 86.2%
trial_failed
False    95167
True     15191
Name: count, dtype: int64

Experimental Set:
set
CSST    110358
Name: count, dtype: int64


In [6]:
## Cell 3: Trial Type Distribution and Success Rates

# Create summary statistics for plotting
trial_summary = df.groupby(['type', 'trial_failed']).size().reset_index(name='count')
trial_summary['outcome'] = trial_summary['trial_failed'].map({False: 'Success', True: 'Failed'})

# Calculate success rates by trial type
success_rates = df.groupby('type').agg({
    'trial_failed': ['count', 'sum', 'mean']
}).round(3)
success_rates.columns = ['total_trials', 'failed_trials', 'failure_rate']
success_rates['success_rate'] = (1 - success_rates['failure_rate']) * 100

print("Success rates by trial type:")
print(success_rates)

# Create the main visualization
plot1 = trial_summary.hvplot.bar(
    x='type', y='count', by='outcome',
    stacked=True,
    title=f'{monkey.title()} - Trial Distribution by Type and Outcome',
    xlabel='Trial Type',
    ylabel='Number of Trials',
    width=600, height=400,
    color=['#2E8B57', '#CD5C5C'],  # Green for success, red for failed
    legend='top_right'
)

plot1


Success rates by trial type:
      total_trials  failed_trials  failure_rate  success_rate
type                                                         
CONT         25294           3933         0.155          84.5
GO           61460           2884         0.047          95.3
STOP         23604           8374         0.355          64.5


:Bars   [type,outcome]   (count)

In [7]:
## Cell 4: Success Rates by Trial Type (Percentage View)
# Create percentage view of success rates
trial_pct = df.groupby('type').apply(
    lambda x: pd.Series({
        'Success': (1 - x['trial_failed'].mean()) * 100,
        'Failed': x['trial_failed'].mean() * 100
    })
).reset_index()

trial_pct_melted = trial_pct.melt(id_vars='type', var_name='outcome', value_name='percentage')

plot2 = trial_pct_melted.hvplot.bar(
    x='type', y='percentage', by='outcome',
    stacked=True,
    title=f'{monkey.title()} - Success Rate by Trial Type (%)',
    xlabel='Trial Type',
    ylabel='Percentage of Trials',
    width=600, height=400,
    color=['#2E8B57', '#CD5C5C'],
    legend='top_right',
    ylim=(0, 100)
)

plot2


/tmp/ipykernel_14310/2379709163.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trial_pct = df.groupby('type').apply(


:Bars   [type,outcome]   (percentage)

In [8]:

## Cell 5: Direction and Trial Type Interaction
# Analyze direction effects across trial types
direction_summary = df.groupby(['type', 'direction', 'trial_failed']).size().reset_index(name='count')
direction_summary['outcome'] = direction_summary['trial_failed'].map({False: 'Success', True: 'Failed'})

# Success rates by type and direction
dir_success = df.groupby(['type', 'direction']).agg({
    'trial_failed': ['count', 'mean']
}).round(3)
dir_success.columns = ['total_trials', 'failure_rate']
dir_success['success_rate'] = (1 - dir_success['failure_rate']) * 100
dir_success = dir_success.reset_index()

print("Success rates by trial type and direction:")
print(dir_success.pivot(index='type', columns='direction', values='success_rate'))

# Visualization
plot3 = direction_summary[direction_summary['outcome'] == 'Success'].hvplot.bar(
    x='type', y='count', by='direction',
    title=f'{monkey.title()} - Successful Trials by Type and Direction',
    xlabel='Trial Type',
    ylabel='Number of Successful Trials',
    width=600, height=400,
    legend='top_right'
)

plot3


Success rates by trial type and direction:
direction     L     R
type                 
CONT       89.6  79.6
GO         95.1  95.6
STOP       58.4  70.6


:Bars   [type,direction]   (count)

In [9]:
## Cell 6: Trial Length Distribution
# Analyze trial length distributions
plot4 = df.hvplot.hist(
    y='trial_length', by='type',
    bins=50, alpha=0.7,
    title=f'{monkey.title()} - Trial Length Distribution by Type',
    xlabel='Trial Length (ms)',
    ylabel='Frequency',
    width=800, height=400,
    legend='top_right'
)

# Add summary statistics
length_stats = df.groupby('type')['trial_length'].describe()
print("Trial length statistics by type (ms):")
print(length_stats.round(1))

plot4


Trial length statistics by type (ms):
        count    mean    std     min     25%     50%     75%     max
type                                                                
CONT  25294.0  2054.4  243.8  1312.0  2076.0  2134.0  2191.0  2251.0
GO    61460.0  2131.4  119.4  1184.0  2093.0  2145.0  2197.0  2251.0
STOP  23604.0  1816.6  107.4   956.0  1749.0  1823.0  1890.0  2099.0


:NdOverlay   [type]
   :Histogram   [trial_length]   (Count)

In [10]:
## Cell 7: Go Cue Timing Analysis
# Analyze go cue timing
plot5 = df.hvplot.hist(
    y='go_cue', by='type',
    bins=50, alpha=0.7,
    title=f'{monkey.title()} - Go Cue Timing Distribution by Type',
    xlabel='Go Cue Time (ms)',
    ylabel='Frequency',
    width=800, height=400,
    legend='top_right'
)

# Summary statistics
go_cue_stats = df.groupby('type')['go_cue'].describe()
print("Go cue timing statistics by type (ms):")
print(go_cue_stats.round(1))

plot5


Go cue timing statistics by type (ms):
        count    mean   std    min    25%     50%     75%     max
type                                                             
CONT  25294.0   999.7  57.6  900.0  950.0   999.0  1049.0  1100.0
GO    61460.0   999.5  58.0  900.0  949.0  1000.0  1049.0  1100.0
STOP  23604.0  1000.0  58.1  900.0  950.0  1000.0  1051.0  1100.0


:NdOverlay   [type]
   :Histogram   [go_cue]   (Count)

In [11]:
## Cell 8: Stop/Continue Signal Delay Analysis
# Filter for STOP and CONTINUE trials only (these have SSD parameters)
signal_trials = df[df['type'].isin(['STOP', 'CONT'])].copy()

print("=== STOP/CONTINUE SIGNAL DELAY ANALYSIS ===")
print(f"\nTotal STOP trials: {len(signal_trials[signal_trials['type'] == 'STOP']):,}")
print(f"Total CONTINUE trials: {len(signal_trials[signal_trials['type'] == 'CONT']):,}")

# Check for missing values
print(f"\nMissing ssd_number: {signal_trials['ssd_number'].isna().sum():,}")
print(f"Missing ssd_len: {signal_trials['ssd_len'].isna().sum():,}")

# Overview of SSD conditions
print(f"\nUnique SSD numbers: {sorted(signal_trials['ssd_number'].dropna().unique())}")
print(f"SSD length range: {signal_trials['ssd_len'].min():.0f} - {signal_trials['ssd_len'].max():.0f} ms")

# SSD number distribution by trial type
ssd_dist = signal_trials.groupby(['type', 'ssd_number']).size().reset_index(name='count')
print(f"\nSSD number distribution by trial type:")
print(ssd_dist.pivot(index='ssd_number', columns='type', values='count').fillna(0))



=== STOP/CONTINUE SIGNAL DELAY ANALYSIS ===

Total STOP trials: 23,604
Total CONTINUE trials: 25,294

Missing ssd_number: 0
Missing ssd_len: 0

Unique SSD numbers: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)]
SSD length range: 24 - 252 ms

SSD number distribution by trial type:
type        CONT  STOP
ssd_number            
1.0         6328  5882
2.0         6189  5890
3.0         6464  5927
4.0         6313  5905


In [12]:
## Cell 9: SSD Length Distribution by Trial Type and SSD Number
# Create a combined grouping variable for better visualization
signal_trials['type_ssd'] = signal_trials['type'] + '_SSD' + signal_trials['ssd_number'].astype(str)

# Plot SSD length distributions with different colors for each SSD number
plot6 = signal_trials.hvplot.hist(
    y='ssd_len', by='type_ssd',
    bins=30, alpha=0.7,
    title=f'{monkey.title()} - Stop/Continue Signal Delay Distribution by SSD Number',
    xlabel='Signal Delay Length (ms)',
    ylabel='Frequency',
    width=800, height=400,
    legend='top_right'
)

# Summary statistics by trial type and SSD number
ssd_detailed_stats = signal_trials.groupby(['type', 'ssd_number'])['ssd_len'].describe()
print("SSD length statistics by trial type and SSD number (ms):")
print(ssd_detailed_stats.round(1))

# Show the unique SSD length for each SSD number (should be consistent)
ssd_mapping = signal_trials.groupby(['type', 'ssd_number'])['ssd_len'].unique()
print(f"\nSSD number to length mapping:")
for (trial_type, ssd_num), lengths in ssd_mapping.items():
    print(f"{trial_type} SSD{ssd_num}: {lengths}")

plot6


SSD length statistics by trial type and SSD number (ms):
                  count   mean   std    min    25%    50%    75%    max
type ssd_number                                                        
CONT 1.0         6328.0   49.7  13.5   24.0   48.0   48.0   48.0   84.0
     2.0         6189.0  108.1  11.9   72.0  108.0  108.0  108.0  132.0
     3.0         6464.0  167.3  10.4  120.0  168.0  168.0  168.0  192.0
     4.0         6313.0  225.4  11.6  168.0  228.0  228.0  228.0  252.0
STOP 1.0         5882.0   49.5  13.6   24.0   48.0   48.0   48.0   84.0
     2.0         5890.0  108.2  12.1   72.0  108.0  108.0  108.0  132.0
     3.0         5927.0  166.9  11.0  120.0  168.0  168.0  168.0  192.0
     4.0         5905.0  225.3  11.8  168.0  228.0  228.0  228.0  252.0

SSD number to length mapping:
CONT SSD1.0: [84 48 24 72]
CONT SSD2.0: [132 108  72  84]
CONT SSD3.0: [180 168 120 144 192]
CONT SSD4.0: [228 168 204 252]
STOP SSD1.0: [84 48 24 72]
STOP SSD2.0: [132 108  72  84]
STOP SSD3.

:NdOverlay   [type_ssd]
   :Histogram   [ssd_len]   (Count)

In [13]:
## Cell 10: SSD Number vs Length Relationship - Violin Plots
# Examine relationship between ssd_number and ssd_len
ssts_df = df[df['type'].isin(['STOP', 'CONT'])].copy()
ssts_df
plot7 = hv.Violin(
    ssts_df, kdims=['ssd_number', 'type'], vdims='ssd_len'
).opts(
    opts.Violin(
        show_legend=True, height=400, width=600,
        violin_color=hv.dim('type').str(),
        legend_position='top_left',
        split='type',
        title=f'{monkey.title()} - SSD Length Distribution by Number and Type',
        xlabel='SSD number',
        ylabel='SSD duration (ms)',
        show_grid=True,
        violin_width=1,
        invert_axes=True
))
plot7

:Violin   [ssd_number,type]   (ssd_len)

In [14]:
## Cell 11: Success Rate by SSD Condition
# Analyze success rates across different SSD conditions
success_by_ssd = signal_trials.groupby(['type', 'ssd_number']).agg({
    'trial_failed': ['count', 'sum', 'mean']
}).round(3)
success_by_ssd.columns = ['total_trials', 'failed_trials', 'failure_rate']
success_by_ssd['success_rate'] = (1 - success_by_ssd['failure_rate']) * 100

print("Success rates by trial type and SSD number:")
print(success_by_ssd)

# Plot success rates as bar plot
success_plot_data = success_by_ssd.reset_index()
plot8 = success_plot_data.hvplot.bar(
    x='ssd_number', y='success_rate', by='type',
    title=f'{monkey.title()} - Success Rate by SSD Number',
    xlabel='SSD Number',
    ylabel='Success Rate (%)',
    width=600, height=400,
    alpha=0.8,
    legend='top_right'
)

plot8

Success rates by trial type and SSD number:
                 total_trials  failed_trials  failure_rate  success_rate
type ssd_number                                                         
CONT 1.0                 6328            949         0.150          85.0
     2.0                 6189            688         0.111          88.9
     3.0                 6464           1289         0.199          80.1
     4.0                 6313           1007         0.160          84.0
STOP 1.0                 5882            606         0.103          89.7
     2.0                 5890           1524         0.259          74.1
     3.0                 5927           2719         0.459          54.1
     4.0                 5905           3525         0.597          40.3


:Bars   [ssd_number,type]   (success_rate)

In [15]:
## Cell 12: SSD Length vs Performance
# Analyze how SSD length affects performance
# Create SSD length bins for analysis
signal_trials['ssd_bin'] = pd.cut(signal_trials['ssd_len'], bins=8, precision=0)

perf_by_length = signal_trials.groupby(['type', 'ssd_bin']).agg({
    'trial_failed': ['count', 'mean']
}).round(3)
perf_by_length.columns = ['trial_count', 'failure_rate']
perf_by_length['success_rate'] = (1 - perf_by_length['failure_rate']) * 100
perf_by_length = perf_by_length.reset_index()

print("Performance by SSD length bins:")
print(perf_by_length[perf_by_length['trial_count'] >= 10])  # Only show bins with sufficient trials

# Aggregate success rates by SSD length for bar plot
success_by_length = signal_trials.groupby(['type', 'ssd_len']).agg({
    'trial_failed': ['count', 'mean']
}).round(3)
success_by_length.columns = ['trial_count', 'failure_rate']
success_by_length['success_rate'] = (1 - success_by_length['failure_rate']) * 100
success_by_length = success_by_length.reset_index()

# Plot as bar chart
plot9 = success_by_length.hvplot.bar(
    x='ssd_len', y='success_rate', 
    by='type',
    title=f'{monkey.title()} - Success Rate by SSD Length',
    xlabel='SSD Length (ms)',
    ylabel='Success Rate (%)',
    width=900, height=400,
    alpha=0.8,
    legend='top_right',
    rot=90,
)

plot9

Performance by SSD length bins:
    type         ssd_bin  trial_count  failure_rate  success_rate
0   CONT    (24.0, 52.0]         5586         0.155          84.5
1   CONT    (52.0, 81.0]          308         0.071          92.9
2   CONT   (81.0, 110.0]         5907         0.111          88.9
3   CONT  (110.0, 138.0]          834         0.133          86.7
4   CONT  (138.0, 166.0]          428         0.096          90.4
5   CONT  (166.0, 195.0]         6065         0.207          79.3
6   CONT  (195.0, 224.0]          470         0.149          85.1
7   CONT  (224.0, 252.0]         5696         0.161          83.9
8   STOP    (24.0, 52.0]         5197         0.094          90.6
9   STOP    (52.0, 81.0]          316         0.149          85.1
10  STOP   (81.0, 110.0]         5549         0.250          75.0
11  STOP  (110.0, 138.0]          838         0.296          70.4
12  STOP  (138.0, 166.0]          451         0.366          63.4
13  STOP  (166.0, 195.0]         5493       

/tmp/ipykernel_14310/2909421133.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  perf_by_length = signal_trials.groupby(['type', 'ssd_bin']).agg({


:Bars   [ssd_len,type]   (success_rate)

In [16]:
# Cell 13: Compute Reaction Time Measures
import ast

print("=== COMPUTING REACTION TIME MEASURES ===")

# Create working copy of the dataframe
df_rt = df.copy()

# Helper function to safely extract saccade start time
def extract_saccade_start(saccade_array):
    """Extract saccade start time from first_relevant_saccade array"""
    
    # Handle None values
    if saccade_array is None:
        return np.nan
    
    # Handle numpy arrays directly
    if isinstance(saccade_array, np.ndarray):
        if len(saccade_array) >= 2:
            return float(saccade_array[0])
        else:
            return np.nan
    
    # Handle lists and tuples
    if isinstance(saccade_array, (list, tuple)):
        if len(saccade_array) >= 2:
            return float(saccade_array[0])
        else:
            return np.nan
    
    # Handle string representations
    if isinstance(saccade_array, str):
        saccade_array = saccade_array.strip()
        if saccade_array == '' or saccade_array == 'nan':
            return np.nan
        try:
            # Try to evaluate string representation of array
            saccade_data = ast.literal_eval(saccade_array)
            if isinstance(saccade_data, (list, tuple)) and len(saccade_data) >= 2:
                return float(saccade_data[0])
            else:
                return np.nan
        except:
            return np.nan
    
    # Handle pandas NA/NaN values
    try:
        if pd.isna(saccade_array):
            return np.nan
    except:
        pass
    
    # If we get here, we couldn't parse it
    return np.nan

# Let's first explore what types we have in the first_relevant_saccade column
print("Exploring first_relevant_saccade column types...")
sample_values = df_rt['first_relevant_saccade'].dropna().iloc[:10]
print("Sample values and their types:")
for i, val in enumerate(sample_values):
    print(f"  {i}: {type(val)} - {str(val)[:100]}...")

# Extract saccade start times
print("\nExtracting saccade start times...")
df_rt['saccade_start'] = df_rt['first_relevant_saccade'].apply(extract_saccade_start)

# Initialize RT columns
df_rt['computed_rt'] = np.nan
df_rt['rt_type'] = ''
df_rt['signal_delay'] = np.nan

# Calculate RTs for each trial
print("Computing reaction times by trial type...")

for idx, row in df_rt.iterrows():
    if pd.notna(row['saccade_start']) and pd.notna(row['go_cue']):
        rt = row['saccade_start'] - row['go_cue']
        
        # Only consider positive RTs (saccade after go cue)
        if rt > 0:
            df_rt.loc[idx, 'computed_rt'] = rt
            
            # Classify RT type based on trial type and outcome
            if row['type'] == 'GO' and not row['trial_failed']:
                df_rt.loc[idx, 'rt_type'] = 'GO_RT'
            elif row['type'] == 'STOP' and row['trial_failed']:
                df_rt.loc[idx, 'rt_type'] = 'Error_Stop_RT'
            elif row['type'] == 'CONT':
                df_rt.loc[idx, 'rt_type'] = 'Continue_RT'
            else:
                df_rt.loc[idx, 'rt_type'] = 'Other'
    
    # Calculate signal delays for STOP and CONT trials
    if row['type'] in ['STOP', 'CONT'] and pd.notna(row['stop_cue']) and pd.notna(row['go_cue']):
        signal_delay = row['stop_cue'] - row['go_cue']
        if signal_delay > 0:  # Ensure signal came after go cue
            df_rt.loc[idx, 'signal_delay'] = signal_delay

# Summary statistics
print("\n=== RT COMPUTATION SUMMARY ===")
rt_summary = df_rt['rt_type'].value_counts()
print("RT types computed:")
print(rt_summary)

print(f"\nValid saccade start times: {df_rt['saccade_start'].notna().sum():,} / {len(df_rt):,}")
print(f"Valid computed RTs: {df_rt['computed_rt'].notna().sum():,}")
print(f"Valid signal delays: {df_rt['signal_delay'].notna().sum():,}")

# RT statistics by type
print("\n=== RT STATISTICS BY TYPE ===")
if df_rt['computed_rt'].notna().sum() > 0:
    rt_stats = df_rt[df_rt['computed_rt'].notna()].groupby('rt_type')['computed_rt'].agg(['count', 'mean', 'std', 'min', 'max']).round(1)
    print(rt_stats)
else:
    print("No valid RTs computed")

# Signal delay statistics
print("\n=== SIGNAL DELAY STATISTICS ===")
if df_rt['signal_delay'].notna().sum() > 0:
    signal_stats = df_rt[df_rt['signal_delay'].notna()].groupby('type')['signal_delay'].agg(['count', 'mean', 'std', 'min', 'max']).round(1)
    print(signal_stats)
else:
    print("No valid signal delays computed")

print("\n=== DATA VALIDATION ===")
# Check for negative RTs (should be rare - anticipatory responses)
negative_rts = ((df_rt['saccade_start'] - df_rt['go_cue']) < 0).sum()
print(f"Negative RTs (anticipatory): {negative_rts}")

# Check for very fast RTs (< 100ms, potentially artifacts)
very_fast_rts = (df_rt['computed_rt'] < 100).sum()
print(f"Very fast RTs (<100ms): {very_fast_rts}")

# Check for very slow RTs (> 1000ms, potentially missed responses)
very_slow_rts = (df_rt['computed_rt'] > 1000).sum()
print(f"Very slow RTs (>1000ms): {very_slow_rts}")

=== COMPUTING REACTION TIME MEASURES ===
Exploring first_relevant_saccade column types...
Sample values and their types:
  0: <class 'numpy.ndarray'> - [1340 1413]...
  1: <class 'numpy.ndarray'> - [1248 1321]...
  2: <class 'numpy.ndarray'> - [1367 1424]...
  3: <class 'numpy.ndarray'> - [1152 1226]...
  4: <class 'numpy.ndarray'> - [1417 1496]...
  5: <class 'numpy.ndarray'> - [1189 1261]...
  6: <class 'numpy.ndarray'> - [1050 1122]...
  7: <class 'numpy.ndarray'> - [1315 1388]...
  8: <class 'numpy.ndarray'> - [1260 1338]...
  9: <class 'numpy.ndarray'> - [1148 1222]...

Extracting saccade start times...
Computing reaction times by trial type...

=== RT COMPUTATION SUMMARY ===
RT types computed:
rt_type
GO_RT            58576
Continue_RT      22902
Other            13250
Error_Stop_RT     8365
                  7265
Name: count, dtype: int64

Valid saccade start times: 103,093 / 110,358
Valid computed RTs: 103,093
Valid signal delays: 48,898

=== RT STATISTICS BY TYPE ===
         

In [17]:
## Cell 14: Stop and Continue Performance by Signal Delay (Figure 1b Replication)
# Replicate Figure 1b: Continue and stop performance
print("=== REPLICATING FIGURE 1B: STOP AND CONTINUE PERFORMANCE ===")

# Filter for STOP and CONT trials with valid signal delays
signal_perf_data = df_rt[df_rt['type'].isin(['STOP', 'CONT']) & df_rt['signal_delay'].notna()].copy()

# Calculate performance by signal delay for each trial type
print(f"Analyzing {len(signal_perf_data):,} trials with valid signal delays")

# For STOP trials: Calculate error rate (percentage of failed stops) by signal delay
stop_performance = signal_perf_data[signal_perf_data['type'] == 'STOP'].groupby('signal_delay').agg({
    'trial_failed': ['count', 'sum', 'mean']
}).round(3)
stop_performance.columns = ['total_trials', 'failed_trials', 'error_rate']
stop_performance['error_percentage'] = stop_performance['error_rate'] * 100
stop_performance = stop_performance.reset_index()

# For CONT trials: Calculate correct rate (percentage of successful continues) by signal delay
cont_performance = signal_perf_data[signal_perf_data['type'] == 'CONT'].groupby('signal_delay').agg({
    'trial_failed': ['count', 'sum', 'mean']
}).round(3)
cont_performance.columns = ['total_trials', 'failed_trials', 'failure_rate']
cont_performance['correct_percentage'] = (1 - cont_performance['failure_rate']) * 100
cont_performance = cont_performance.reset_index()

print("STOP trial error rates by signal delay:")
print(stop_performance[['signal_delay', 'total_trials', 'error_percentage']])

print("\nCONT trial success rates by signal delay:")
print(cont_performance[['signal_delay', 'total_trials', 'correct_percentage']])

# Create the plot replicating Figure 1b
stop_plot = stop_performance.hvplot.line(
    x='signal_delay', y='error_percentage',
    color='red', line_width=3, 
    label=f'Error stop ({monkey.title()})',
    markers=True, marker_size=8
)

cont_plot = cont_performance.hvplot.line(
    x='signal_delay', y='correct_percentage', 
    color='blue', line_width=3,
    label=f'Correct continue ({monkey.title()})',
    markers=True, marker_size=8, line_dash='dashed'
)

# Combine plots
plot_fig1b = (stop_plot * cont_plot).opts(
    title=f'{monkey.title()} - Stop and Continue Performance (Figure 1b)',
    xlabel='Stop/continue signal delay (ms)',
    ylabel='Percentage of saccades',
    width=600, height=400,
    ylim=(0, 100),
    legend_position='right',
    show_grid=True
)

print(f"\n=== RACE MODEL PREDICTIONS ===")
print(f"Error stop rates should INCREASE with longer SSDs (race model prediction)")
print(f"Continue success should be relatively STABLE across CSDs")

# Check race model predictions
ssd_range = stop_performance['signal_delay'].max() - stop_performance['signal_delay'].min()
error_range = stop_performance['error_percentage'].max() - stop_performance['error_percentage'].min()

print(f"\nSSD range: {ssd_range:.0f} ms")
print(f"Error rate change: {error_range:.1f} percentage points")

if error_range > 20:  # Arbitrary threshold for "substantial" increase
    print("✓ Error rates show substantial increase with SSD (consistent with race model)")
else:
    print("? Error rates show modest increase with SSD")

cont_range = cont_performance['correct_percentage'].max() - cont_performance['correct_percentage'].min()
print(f"Continue success variability: {cont_range:.1f} percentage points")

if cont_range < 20:  # Arbitrary threshold for "stable"
    print("✓ Continue success rates relatively stable (consistent with preserved saccade generation)")
else:
    print("? Continue success shows notable variability")

plot_fig1b

=== REPLICATING FIGURE 1B: STOP AND CONTINUE PERFORMANCE ===
Analyzing 48,898 trials with valid signal delays
STOP trial error rates by signal delay:
    signal_delay  total_trials  error_percentage
0           24.0           579               7.8
1           48.0          4618               9.6
2           72.0           316              14.9
3           84.0           962              18.5
4          108.0          4587              26.4
5          120.0           128              32.0
6          132.0           710              29.2
7          144.0           451              36.6
8          168.0          4783              47.3
9          180.0           553              50.3
10         192.0           157              22.9
11         204.0           456              45.2
12         228.0          5162              62.0
13         252.0           142              37.3

CONT trial success rates by signal delay:
    signal_delay  total_trials  correct_percentage
0           24.0     

:Overlay
   .Curve.Error_stop_left_parenthesis_Fiona_right_parenthesis       :Curve   [signal_delay]   (error_percentage)
   .Curve.Correct_continue_left_parenthesis_Fiona_right_parenthesis :Curve   [signal_delay]   (correct_percentage)